In [1]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("https://github.com/maheshkshi400/MK").getOrCreate()




ModuleNotFoundError: No module named 'pyspark'

In [6]:
array_data=[
    ("John",4,1),
    ("John",6,2),
    ("David",7,3),
    ("mike",3,4),
    ("David",5,2),
    ("John",7,3),
    ("john",9,7),
    ("David",5,4),
    ("mike",7,5),
    ("john",7,6),
    ("David",4,3),
    ("mike",4,7),
    ("David",5,4),
    ("john",6,4),
    ("David",8,2)
]

array_schema=["Name","score_1","score_2"]

df1= spark.createDataFrame(array_data,array_schema)
df1.show()

+-----+-------+-------+
| Name|score_1|score_2|
+-----+-------+-------+
| John|      4|      1|
| John|      6|      2|
|David|      7|      3|
| mike|      3|      4|
|David|      5|      2|
| John|      7|      3|
| john|      9|      7|
|David|      5|      4|
| mike|      7|      5|
| john|      7|      6|
|David|      4|      3|
| mike|      4|      7|
|David|      5|      4|
| john|      6|      4|
|David|      8|      2|
+-----+-------+-------+



In [8]:
# convert sample dataframe into Array dataframe

from pyspark.sql import functions as F

master_df=df1.groupBy("Name").agg(F.collect_list('score_1').alias('Array_score_1'),
                                  F.collect_list('score_2').alias('Array_score_2'))

master_df.printSchema()
master_df.show()


root
 |-- Name: string (nullable = true)
 |-- Array_score_1: array (nullable = false)
 |    |-- element: long (containsNull = false)
 |-- Array_score_2: array (nullable = false)
 |    |-- element: long (containsNull = false)

+-----+------------------+------------------+
| Name|     Array_score_1|     Array_score_2|
+-----+------------------+------------------+
| John|         [4, 6, 7]|         [1, 2, 3]|
|David|[7, 5, 5, 4, 5, 8]|[3, 2, 4, 3, 4, 2]|
| mike|         [3, 7, 4]|         [4, 5, 7]|
| john|         [9, 7, 6]|         [7, 6, 4]|
+-----+------------------+------------------+



In [10]:
# Apply array_zip function on Array Df

arr_zip_df=master_df.withColumn("Zipped_value",F.arrays_zip("Array_score_1","Array_score_2"))

arr_zip_df.show(truncate=False)

+-----+------------------+------------------+------------------------------------------------+
|Name |Array_score_1     |Array_score_2     |Zipped_value                                    |
+-----+------------------+------------------+------------------------------------------------+
|John |[4, 6, 7]         |[1, 2, 3]         |[{4, 1}, {6, 2}, {7, 3}]                        |
|David|[7, 5, 5, 4, 5, 8]|[3, 2, 4, 3, 4, 2]|[{7, 3}, {5, 2}, {5, 4}, {4, 3}, {5, 4}, {8, 2}]|
|mike |[3, 7, 4]         |[4, 5, 7]         |[{3, 4}, {7, 5}, {4, 7}]                        |
|john |[9, 7, 6]         |[7, 6, 4]         |[{9, 7}, {7, 6}, {6, 4}]                        |
+-----+------------------+------------------+------------------------------------------------+



In [13]:
empdf= [
    ('Sales_dept',[{'emp_name':'Ganesh','salary':'8000','yrs_of_service':'10','age':'35'},
                   {'emp_name':'Suresh','salary':'7000','yrs_of_service':'7','age':'33'},
                   {'emp_name':'Prachi','salary':'10000','yrs_of_service':'5','age':'37'},
                   {'emp_name':'Minakshi','salary':'6000','yrs_of_service':'2','age':'23'},
                   {'emp_name':'Kshitija','salary':'8000','yrs_of_service':'6','age':'31'}]),
    ('HR_dept',[{'emp_name':'shivani','salary':'10000','yrs_of_service':'5','age':'35'},
                   {'emp_name':'Makrand','salary':'9000','yrs_of_service':'6','age':'37'},
                   {'emp_name':'Swati','salary':'12000','yrs_of_service':'8','age':'42'},
                   {'emp_name':'Girija','salary':'15000','yrs_of_service':'12','age':'42'},
                   {'emp_name':'Mahadev','salary':'40000','yrs_of_service':'15','age':'45'}])
]

df_brand = spark.createDataFrame(empdf,schema=['Depatement','Employee'])
df_brand.printSchema()
df_brand.show(truncate=False)


root
 |-- Depatement: string (nullable = true)
 |-- Employee: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Depatement|Employee                                                                                                                                                                                                                                                                                                                                                              |
+----------+-----------------------------

In [15]:
# Apply Array_zip

df_brandzip = df_brand.withColumn("zip",F.arrays_zip(df_brand["Employee"]))
df_brandzip.show(truncate=False)


+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Depatement|Employee                                                                                                                                                                                                                                               

In [20]:
# Apply Explode

df_brand_exp = df_brandzip.withColumn("Explode",F.explode(df_brandzip.zip))
df_brand_exp.show(truncate=False)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------+
|Depatement|Employee                                                                                                                                                                     

In [23]:
df_brand_output = df_brand_exp.withColumn("employee_emp_name",df_brand_exp['Explode.Employee.emp_name'])\
                            .withColumn("employee_yrs_of_service",df_brand_exp['Explode.Employee.yrs_of_service'])\
                            .withColumn("employee_salary",df_brand_exp['Explode.Employee.salary'])\
                            .withColumn("employee_age",df_brand_exp['Explode.Employee.age']).drop("Explode").drop("zip").drop("Employee")

df_brand_output.show()

+----------+-----------------+-----------------------+---------------+------------+
|Depatement|employee_emp_name|employee_yrs_of_service|employee_salary|employee_age|
+----------+-----------------+-----------------------+---------------+------------+
|Sales_dept|           Ganesh|                     10|           8000|          35|
|Sales_dept|           Suresh|                      7|           7000|          33|
|Sales_dept|           Prachi|                      5|          10000|          37|
|Sales_dept|         Minakshi|                      2|           6000|          23|
|Sales_dept|         Kshitija|                      6|           8000|          31|
|   HR_dept|          shivani|                      5|          10000|          35|
|   HR_dept|          Makrand|                      6|           9000|          37|
|   HR_dept|            Swati|                      8|          12000|          42|
|   HR_dept|           Girija|                     12|          15000|      